# Load Essential Libraries

In [30]:
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn.utils.multiclass import unique_labels
import pandas as pd
import torch
import torchvision
import torchaudio
import pickle
import json
print(torch.__version__)
print(torchvision.__version__)
print(torchaudio.__version__)

1.12.1
0.13.1
0.12.1


In [31]:
input_file = '/mnt/nas/NTCIR17/FinARG/ECC_Argument Relation/ian/TEST/MODELS/bart_probs.pkl'
with open(input_file, 'rb') as f:
    bart_probs = pickle.load(f)

bart_probs

array([[9.4555231e-05, 9.9989319e-01, 1.2210530e-05],
       [1.4233534e-04, 9.9983799e-01, 1.9669387e-05],
       [8.9063078e-02, 3.2450467e-02, 8.7848645e-01],
       ...,
       [1.7493308e-02, 1.3496364e-03, 9.8115706e-01],
       [9.9993974e-01, 2.6163338e-05, 3.4060144e-05],
       [3.3560437e-01, 6.6214317e-01, 2.2524309e-03]], dtype=float32)

In [32]:
input_file = '/mnt/nas/NTCIR17/FinARG/ECC_Argument Relation/ian/TEST/MODELS/deberta_probs.pkl'
with open(input_file, 'rb') as f:
    deberta_probs = pickle.load(f)

deberta_probs

array([[9.8293670e-04, 9.9823707e-01, 7.8000751e-04],
       [3.8603824e-03, 9.9509704e-01, 1.0426191e-03],
       [3.6445118e-03, 9.9426717e-01, 2.0882527e-03],
       ...,
       [6.0849190e-03, 6.4562015e-02, 9.2935306e-01],
       [9.9952620e-01, 3.0919875e-04, 1.6464289e-04],
       [1.2641157e-03, 9.9717045e-01, 1.5654155e-03]], dtype=float32)

In [33]:
llr_probs = pd.read_csv('/mnt/nas/NTCIR17/FinARG/ECC_Argument Relation/ian/TEST/MODELS/LLRDNN_result_Smote_for_ian_100final.csv').to_numpy()

llr_probs

array([[1.04339860e-01, 8.95047903e-01, 6.12245000e-04],
       [5.17025590e-01, 4.78043973e-01, 4.93049100e-03],
       [9.99996781e-01, 6.87000000e-07, 2.56000000e-06],
       ...,
       [8.65728560e-02, 9.13373649e-01, 5.34000000e-05],
       [5.17025590e-01, 4.78043973e-01, 4.93049100e-03],
       [5.17025590e-01, 4.78043973e-01, 4.93049100e-03]])

# Find the Weights

In [ ]:
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report

def find_best_weights(predictions_model1, predictions_model2, y_true):
    best_f1_score = 0.0
    best_weight_model1 = 0.0
    best_weight_model2 = 0.0
    best_weighted_predictions = None

    for weight_model1 in np.arange(0, 1.1, 0.000001):
        weight_model2 = 1.0 - weight_model1

        weighted_predictions = vote(predictions_model1, predictions_model2, weight_model1, weight_model2)
        weighted_predictions = weighted_predictions.detach().numpy().argmax(axis=-1)

        f1_score = metrics.f1_score(y_true, weighted_predictions, average='macro')

        if f1_score > best_f1_score:
            best_f1_score = f1_score
            best_weight_model1 = weight_model1
            best_weight_model2 = weight_model2
            best_weighted_predictions = weighted_predictions

    return best_weighted_predictions, best_weight_model1, best_weight_model2, best_f1_score

predictions_model1 = LLRDNN_ans  
predictions_model2 = BART_ans  
y_true = LLRDNN['labels']  

# 寻找最佳权重
best_weighted_predictions, best_weight_model1, best_weight_model2, best_f1_score = find_best_weights(predictions_model1, predictions_model2, y_true)

print('Best weights: Model 1 - {0}, Model 2 - {1}'.format(best_weight_model1, best_weight_model2))
print('Best Macro F1-score: {0}'.format(best_f1_score))

y_predict = best_weighted_predictions

print('Macro-average: {0}'.format(metrics.f1_score(y_true, y_predict, average = 'macro')))
print('Micro-average: {0}'.format(metrics.f1_score(y_true, y_predict, average = 'micro')))
print(classification_report(y_true, y_predict, digits=6))

print(metrics.confusion_matrix(y_true, y_predict))

# LLR_DNN(RS) & BART(CL)​

In [34]:
def vote(predictions1, predictions2, weight1, weight2):
    if not isinstance(predictions1, torch.Tensor):
        predictions1 = torch.tensor(predictions1)
    if not isinstance(predictions2, torch.Tensor):
        predictions2 = torch.tensor(predictions2)
    # 對機率分布加權
    if predictions1.dim() == 2 and predictions2.dim() == 2:
        weighted_predictions = weight1 * predictions1 + weight2 * predictions2

    # 加權投票
    elif predictions1.dim() == 1 and predictions2.dim() == 1:
        num_classes = max(predictions1.max().item(), predictions2.max().item()) + 1
        weighted_predictions = torch.zeros_like(predictions1)
        for i in range(num_classes):
            weighted_predictions[(predictions1 == i) & (predictions2 == i)] = i
            weighted_predictions[(predictions1 == i) & (predictions2 != i)] = i if weight1 >= weight2 else i
            weighted_predictions[(predictions1 != i) & (predictions2 == i)] = i if weight1 < weight2 else i

    else:
        raise ValueError("Invalid input shape. Predictions should be either 1D tensors or 2D tensors.")

    return weighted_predictions

In [35]:
weight_model1 = 0.44857
weight_model2 = 0.55143

predictions_model1 = llr_probs  
predictions_model2 = bart_probs  

weighted_predictions = vote(predictions_model1, predictions_model2, weight_model1, weight_model2)
weighted_predictions = weighted_predictions.detach().numpy().argmax(axis=-1)

y_predict = pd.DataFrame(weighted_predictions)
label_counts = y_predict[0].value_counts()
print(label_counts)
print('=========================')
print(y_predict)

1    505
0    181
2      4
Name: 0, dtype: int64
     0
0    1
1    1
2    0
3    0
4    0
..  ..
685  0
686  1
687  2
688  0
689  1

[690 rows x 1 columns]


In [20]:
# Read the JSON file
with open('../ECC_Argument_Relation_Test.json') as file:
    data = json.load(file)

# Assuming you have the DataFrame named "combined_df" with 'soft_vote_preds' column
for item in data:
    test_id = item['TestID']
    item['Prediction'] = int(y_predict.loc[test_id])

# Save the updated JSON data to a new file
output_file = 'ECC_Argument_Classification_Test_top 4 hard.json'
with open(output_file, 'w') as file:
    json.dump(data, file)

print("JSON File Predictions Completed.")

JSON File Predictions Completed.


In [7]:
df = pd.read_csv("../ECCAR.csv")
df['Prediction'] = y_predict
df

,TestID,sentence1,sentence2,Prediction
0,0,"We now have 18 planes in service for Amazon, a...","So on that front, it's better control, better ...",1
1,1,So we are not afraid to bring in new capabilit...,And we'll continue to do that because I think ...,1
2,2,"This is a young and rapidly growing business, ...","We're excited about the potential there, and r...",0
3,3,So we are always thinking at least of the tech...,"Right, I would say actually we've also seen so...",0
4,4,And as we continue to improve the margin profi...,That's the guidance we gave.,0
...,...,...,...,...
685,685,"But that, I think, is the primary focus for th...","Over the longer term, I think as that works, p...",0
686,686,It's worth noting that video ads take place on...,Video ads are really exciting.,1
687,687,There will be variability quarter to quarter.,"But overall, when I look at what is it that we...",2
688,688,And so obviously at some point if those techno...,"We price for the value that we're delivering, ...",0


In [8]:
print(df['sentence1'][5])
print(df['sentence2'][5])

Now over the next period, especially in H1, we continue to see -- expect to see very good hybrid percentage and execution, which help to offset some of that.
Heather, let me -- on gross margin, over a long-term period and intelligent cloud, given the expansive TAM and the growth opportunity we see, it will create gross margin pressure over long-term.


In [9]:
label_counts = df['Prediction'].value_counts()
print(label_counts)

1    505
0    181
2      4
Name: Prediction, dtype: int64


In [10]:
df.to_json('./submission/ECCAR_1.json', orient='records')

# LLR_DNN(RS) & BART(CL) & DEBERTA(CL)

In [21]:
def vote(predictions1, predictions2, predictions3, weight1, weight2, weight3):
    if not isinstance(predictions1, torch.Tensor):
        predictions1 = torch.tensor(predictions1)
    if not isinstance(predictions2, torch.Tensor):
        predictions2 = torch.tensor(predictions2)
    if not isinstance(predictions3, torch.Tensor):
        predictions3 = torch.tensor(predictions3)
    
    # 對機率分布加權
    if predictions1.dim() == 2 and predictions2.dim() == 2 and predictions3.dim() == 2:
        weighted_predictions = weight1 * predictions1 + weight2 * predictions2 + weight3 * predictions3
    
    # 加權投票
    elif predictions1.dim() == 1 and predictions2.dim() == 1 and predictions3.dim() == 1:
        num_classes = max(predictions1.max().item(), predictions2.max().item(), predictions3.max().item()) + 1
        weighted_predictions = torch.zeros_like(predictions1)
        for i in range(num_classes):
            weighted_predictions[(predictions1 == i) & (predictions2 == i) & (predictions3 == i)] = i
            weighted_predictions[(predictions1 == i) & (predictions2 != i) & (predictions3 != i)] = i if weight1 >= 0 and weight1 >= 0 else i
            weighted_predictions[(predictions1 != i) & (predictions2 == i) & (predictions3 != i)] = i if weight2 >= 0 and weight2 >= 0 else i
            weighted_predictions[(predictions1 != i) & (predictions2 != i) & (predictions3 == i)] = i if weight3 >= 0 and weight3 >= 0 else i
            weighted_predictions[(predictions1 == i) & (predictions2 == i) & (predictions3 != i)] = i if weight1 + weight2 >= 0 else i
            weighted_predictions[(predictions1 == i) & (predictions2 != i) & (predictions3 == i)] = i if weight1 + weight3 >= 0 else i
            weighted_predictions[(predictions1 != i) & (predictions2 == i) & (predictions3 == i)] = i if weight2 + weight3 >= 0 else i
    
    else:
        raise ValueError("Invalid input shape. Predictions should be either 1D tensors or 2D tensors.")

    return weighted_predictions

In [22]:
weight_model1 = 0.4
weight_model2 = 0.52
weight_model3 = 0.08

predictions_model1 = llr_probs  
predictions_model2 = bart_probs 
predictions_model3 = deberta_probs

weighted_predictions = vote(predictions_model1, predictions_model2, predictions_model3, weight_model1, weight_model2, weight_model3)
weighted_predictions = weighted_predictions.detach().numpy().argmax(axis=-1)

y_predict = pd.DataFrame(weighted_predictions)
label_counts = y_predict[0].value_counts()
print(label_counts)
print('=========================')
print(y_predict)

1    505
0    180
2      5
Name: 0, dtype: int64
     0
0    1
1    1
2    2
3    0
4    0
..  ..
685  0
686  1
687  2
688  0
689  1

[690 rows x 1 columns]


In [23]:
df = pd.read_csv("../ECCAR.csv")
df['Prediction'] = y_predict
df

,TestID,sentence1,sentence2,Prediction
0,0,"We now have 18 planes in service for Amazon, a...","So on that front, it's better control, better ...",1
1,1,So we are not afraid to bring in new capabilit...,And we'll continue to do that because I think ...,1
2,2,"This is a young and rapidly growing business, ...","We're excited about the potential there, and r...",2
3,3,So we are always thinking at least of the tech...,"Right, I would say actually we've also seen so...",0
4,4,And as we continue to improve the margin profi...,That's the guidance we gave.,0
...,...,...,...,...
685,685,"But that, I think, is the primary focus for th...","Over the longer term, I think as that works, p...",0
686,686,It's worth noting that video ads take place on...,Video ads are really exciting.,1
687,687,There will be variability quarter to quarter.,"But overall, when I look at what is it that we...",2
688,688,And so obviously at some point if those techno...,"We price for the value that we're delivering, ...",0


In [24]:
# Read the JSON file
with open('../ECC_Argument_Relation_Test.json') as file:
    data = json.load(file)

# Assuming you have the DataFrame named "combined_df" with 'soft_vote_preds' column
for item in data:
    test_id = item['TestID']
    item['Prediction'] = int(y_predict.loc[test_id])

# Save the updated JSON data to a new file
output_file = 'ECC_Argument_Classification_Test_top 4 hard.json'
with open(output_file, 'w') as file:
    json.dump(data, file)

print("JSON File Predictions Completed.")

JSON File Predictions Completed.


In [25]:
label_counts = df['Prediction'].value_counts()
print(label_counts)

1    505
0    180
2      5
Name: Prediction, dtype: int64


In [15]:
df.to_json('./submission/ECCAR_2.json', orient='records')

# BART(CL)

In [26]:
y_predict = pd.DataFrame(bart_probs.argmax(axis=-1))
label_counts = y_predict[0].value_counts()
print(label_counts)
print('=========================')
print(y_predict)

1    506
0    177
2      7
Name: 0, dtype: int64
     0
0    1
1    1
2    2
3    0
4    0
..  ..
685  0
686  1
687  2
688  0
689  1

[690 rows x 1 columns]


In [27]:
df = pd.read_csv("../ECCAR.csv")
df['Prediction'] = y_predict
df

,TestID,sentence1,sentence2,Prediction
0,0,"We now have 18 planes in service for Amazon, a...","So on that front, it's better control, better ...",1
1,1,So we are not afraid to bring in new capabilit...,And we'll continue to do that because I think ...,1
2,2,"This is a young and rapidly growing business, ...","We're excited about the potential there, and r...",2
3,3,So we are always thinking at least of the tech...,"Right, I would say actually we've also seen so...",0
4,4,And as we continue to improve the margin profi...,That's the guidance we gave.,0
...,...,...,...,...
685,685,"But that, I think, is the primary focus for th...","Over the longer term, I think as that works, p...",0
686,686,It's worth noting that video ads take place on...,Video ads are really exciting.,1
687,687,There will be variability quarter to quarter.,"But overall, when I look at what is it that we...",2
688,688,And so obviously at some point if those techno...,"We price for the value that we're delivering, ...",0


In [28]:
label_counts = df['Prediction'].value_counts()
print(label_counts)

1    506
0    177
2      7
Name: Prediction, dtype: int64


In [29]:
# Read the JSON file
with open('../ECC_Argument_Relation_Test.json') as file:
    data = json.load(file)

# Assuming you have the DataFrame named "combined_df" with 'soft_vote_preds' column
for item in data:
    test_id = item['TestID']
    item['Prediction'] = int(y_predict.loc[test_id])

# Save the updated JSON data to a new file
output_file = 'ECC_Argument_Classification_Test_top 4 hard.json'
with open(output_file, 'w') as file:
    json.dump(data, file)

print("JSON File Predictions Completed.")

JSON File Predictions Completed.


In [9]:
df.to_json('./submission/ECCAR_3.json', orient='records')

In [36]:
import pandas as pd
ECCAR_1 = pd.read_json('./TMUNLP_ECCAR_1.json')
ECCAR_2 = pd.read_json('./TMUNLP_ECCAR_2.json')
ECCAR_3 = pd.read_json('./TMUNLP_ECCAR_3.json')


In [39]:
label_counts = ECCAR_1['Prediction'].value_counts()
print(label_counts)

1    505
0    180
2      5
Name: Prediction, dtype: int64


In [40]:
label_counts = ECCAR_2['Prediction'].value_counts()
print(label_counts)

1    505
0    181
2      4
Name: Prediction, dtype: int64


In [41]:
label_counts = ECCAR_3['Prediction'].value_counts()
print(label_counts)

1    506
0    177
2      7
Name: Prediction, dtype: int64
